# HOUSING KAGGLE REGRESSION PROBLEM






### First regression model
OOTB Random Forest Regressor to be used as our ruler

In [0]:
# loading dataset into the script
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_train.csv')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [0]:
# a generic method to arrange the variables
housing = df.copy()

# starting by filling the blank spaces 
for column in housing.columns:
    if housing[column].dtype != 'object':
        housing[column].fillna(housing[column].mean(), inplace= True)
    else:
        housing[column] = housing[column].astype('str')
        housing[column].fillna(housing[column].mode(), inplace= True)

In [0]:
## fitting the first regression model 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score

# splitting the dependent (predictor) and dependent (target) features
x = housing.drop(['SalePrice'], axis= 1)
y = housing['SalePrice']

# applying label encoding
le = LabelEncoder()
for column in x.columns:
    if x[column].dtype == 'object':
        x[column] = le.fit_transform(x[column])

# splitting the data into train and test
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size= 0.2, random_state= 55)

In [84]:
# calling the regressor
reg1 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

# fitting the model
reg1.fit(x_treino, y_treino)

# making the prediction of target in trein set
prev = reg1.predict(x_teste)


# error evaluation
from sklearn.metrics import mean_squared_log_error
import numpy as np


erro = np.sqrt(mean_squared_log_error(y_teste, prev))
erro

0.13568714490519806

In [0]:
# making the predictions over submission set
# loading submission data
url_data = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_test.csv'
data = pd.read_csv(url_data)

In [0]:
# filling in the gaps in the same manner as used in the trein set
for column in data.columns:
    if data[column].dtype != 'object':
        data[column].fillna(data[column].mean(), inplace= True)
    else:
        data[column] = data[column].astype('str')
        data[column].fillna(data[column].mode(), inplace= True)

# using label encoder over the test set the same way as used in trein set 
le = LabelEncoder()
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le.fit_transform(data[column])

In [0]:
# df for kaggle evaluation
final = reg1.predict(data)
envio = pd.DataFrame({'Id': data['Id'], 'SalePrice': final})

In [0]:
# saving the df in a csv to be uploaded

#path = r'C:\Users\Avell\Desktop\Python\github\datasets'
#nome = '\housing_submission_1.csv'
#envio.to_csv(path+nome, index= False)

Of corse we didn't expect here anything extremely accurate for some reasons:
* The model itself was implemented just out of the box
* The approach to fill the missing values using mode to categorical features and mean in the case of numerical ones didn't take in account any exploratory analysis of the variables
* No methodology for feature engineering and in order to tune model precision by finding cerrelations or opportunities to dimensional reduction
* All categorical features were encoded with the same method

## Model improvement attempts
### Second regression model scoring rmsle 0.1466

In [0]:
# checando com o feature importance
featimp = pd.Series(reg1.feature_importances_, index= x_teste.columns).sort_values(ascending=False)
featimp.index
feat_upper = featimp.index[0: 24]

In [0]:
# novo x
x2 = x.copy()
y2 = y.copy()

# novo encoding
le2 = LabelEncoder()
for column in x2.columns:
    if x2[column].dtype == 'object':
        x2[column] = le2.fit_transform(x2[column])

In [91]:
# chamando o modelo
reg2 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

x2_treino, x2_teste, y2_treino, y2_teste = train_test_split(x2, y2, test_size= 0.2, random_state= 11)

# ajustando um modelo
reg2.fit(x2_treino[feat_upper], y2_treino)

# fazendo a previsao
prev2 = reg2.predict(x2_teste[feat_upper])


# avaliando o erro
from sklearn.metrics import mean_squared_log_error
import numpy as np


error2 = np.sqrt(mean_squared_log_error(y2_teste, prev2))
error2


0.1443646655720345

In [92]:
y_axis = []
x_axis = []

for m in(range(len(featimp.index))):

  error2_ = 0
  n = 5
  
  for i in(range(n)):
    # chamando o modelo
    reg2 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
              max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=5,
              oob_score=False, random_state=None, verbose=0, warm_start=False)

    x2_treino, x2_teste, y2_treino, y2_teste = train_test_split(x2, y2, test_size= 0.2, random_state= i**2)

    # ajustando um modelo
    reg2.fit(x2_treino[featimp.index[0: m+1]], y2_treino)

    # fazendo a previsao
    prev2 = reg2.predict(x2_teste[featimp.index[0: m+1]])


    # avaliando o erro
    from sklearn.metrics import mean_squared_log_error
    import numpy as np


    error2 = np.sqrt(mean_squared_log_error(y2_teste, prev2))
    #print(error2)
    error2_ += error2

  y_axis.append(error2_/n)
  x_axis.append(m)
  print(m, error2_/n)

0 0.2213524143735138
1 0.21525065706019353
2 0.18195680956610813
3 0.170858201521555
4 0.16722131205634874
5 0.161737196146741
6 0.16020528201318013
7 0.15922655647518869
8 0.157491264310203
9 0.1509195760680191
10 0.15201801656423045
11 0.1526665808129707
12 0.14843617273024728
13 0.1459006181914338
14 0.1473665677295048
15 0.14579156812940736
16 0.1471499895719634
17 0.1461735872676941
18 0.14674172001927394
19 0.14512919901430196
20 0.146947171763449
21 0.14715363964461173
22 0.14509171321130637
23 0.14580995941881342
24 0.1459527078104604
25 0.14328048330596235
26 0.14217048733796375
27 0.1421051062370437
28 0.1425451335558059
29 0.1414952015536075
30 0.14265805207548604
31 0.14192043189068676
32 0.14142116805433805
33 0.14161825106388087
34 0.14167671347488375
35 0.14082550781521908
36 0.14104190571099692
37 0.14195474591108403
38 0.14146467561843928
39 0.14231126069605643


KeyboardInterrupt: ignored

In [0]:
error_df = pd.DataFrame({'n of features': x_axis,
                         'error': y_axis}).sort_values(by= ['error'], ascending= True)

error_df

In [0]:
import matplotlib.pyplot as plt

plt.title('RMSLE')
plt.xlabel('Number of sorted features')
plt.ylabel('Error')
plt.plot(x_axis, y_axis)
plt.show()

In [0]:
dataframe = pd.read_csv('https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_train.csv')

In [0]:
# exploring data 

for column in dataframe.columns:
  if dataframe[column].dtype == 'object':
    print(dataframe[column].value_counts())

In [0]:
featimp.index